In [2]:
###Main page: PubmedApp

from flask import Flask, render_template, request, redirect
from bokeh.embed import components
#import PubDatePlotting as pdp
#import StateGraph as sg

# output_notebook()

app = Flask(__name__)

app.vars = {}

#setting up navigation info

app.nav_id = {'counts':'counts','geo':'geo','auth_inst':'auth_inst'}
app.nav_name = {'counts':'Raw Counts','geo':'Geographic','auth_inst':'Authors & Institutions'}


@app.route('/', methods=['GET', 'POST'])
def index():
    # nquestions=app_lulu.nquestions
    if request.method == 'GET':
        return render_template('pubsearch.html')
    else:
        
        del app.vars

#         app.vars['cancertype'] = request.form['cancertype']

#         p1 = pdp.pubByDate(app.vars['cancertype'], False)
#         p2 = pdp.pubByDate(app.vars['cancertype'], True)
#         p3, p4 = stateGraph(app.vars['cancertype'])
#         plots = {'p1': p1, 'p2': p2, 'p3': p3, 'p4': p4}
#         script, div = components(plots)

        #return render_template('pubsearch.html', script=script, div=div, ctype=app.vars['cancertype'])
        return render_template('pubsearch.html')



@app.route('/search', methods=['GET', 'POST'])
def searchView():
    if request.method == 'GET':
        app.vars = {}
        return render_template('pubsearch.html')
    else:
        app.vars = {}
        app.vars['searchStr'] = request.form['searchterm']
        return countsView()


        
@app.route('/counts', methods=['POST'])
def countsView():
    app.curpage = "counts"
    if 'countsData' not in app.vars:
        
        app.vars['countsData'] = True
        
#         while True:
#             try:
#                 app.yearplot = yearGraph(app.vars['searchStr'])
#                 break
#             except:
#                 print("Error getting year data")
        app.yearplot = yearGraph(app.vars['searchStr'])
        plots = {'yearplot': app.yearplot}
        script, div = components(plots)
        ######render
        return render_template('pubview.html', searchstring=app.vars['searchStr'], script=script, div=div, curpage=app.curpage, nav_id=app.nav_id, nav_name=app.nav_name)
    else:
        
        
        plots = {'yearplot': app.yearplot}
        script, div = components(plots)
        
        ######render
        return render_template('pubview.html', searchstring=app.vars['searchStr'], script=script, div=div, curpage=app.curpage, nav_id=app.nav_id, nav_name=app.nav_name)

    
    
    
    
@app.route('/geo', methods=['POST'])
def geoView():
    app.curpage = "geo"
    if 'geoData' not in app.vars:
        app.vars['geoData'] = True
        
        #connect to Pubmed and try to get data, if fail restart
#         while True:
#             try:
#                 app.stateplot, app.stateplotnorm = stateGraph(app.vars['searchStr'])
#                 break
#             except:
#                 print("Error getting state data")
                
        app.stateplot, app.stateplotnorm = stateGraph(app.vars['searchStr'])
        plots = {'stateplot': app.stateplot, 'stateplotnorm': app.stateplotnorm}
        script, div = components(plots)
        
        ######render
        return render_template('pubview.html', searchstring=app.vars['searchStr'], script=script, div=div, curpage=app.curpage, nav_id=app.nav_id, nav_name=app.nav_name)
    else:
        
        plots = {'stateplot': app.stateplot, 'stateplotnorm': app.stateplotnorm}
        script, div = components(plots)
        
        ######render
        return render_template('pubview.html', searchstring=app.vars['searchStr'], script=script, div=div, curpage=app.curpage, nav_id=app.nav_id, nav_name=app.nav_name)

    
    
    
@app.route('/auth_inst', methods=['POST'])
def auth_instView():
    app.curpage = "auth_inst"
    if 'auth_instData' not in app.vars:
        app.vars['auth_instData'] = True
        
        ######render
        return render_template('pubview.html', searchstring=app.vars['searchStr'], curpage=app.curpage, nav_id=app.nav_id, nav_name=app.nav_name)
    else:
        
        ######render
        return render_template('pubview.html', searchstring=app.vars['searchStr'], curpage=app.curpage, nav_id=app.nav_id, nav_name=app.nav_name)


    
if __name__ == "__main__":
    app.run(port=33507)

 * Running on http://127.0.0.1:33507/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2017 15:03:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2017 15:03:00] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2017 15:03:00] "GET /static/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2017 15:03:00] "GET /favicon.ico HTTP/1.1" 404 -
[2017-05-25 15:03:02,725] ERROR in app: Exception on /search [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/anaconda/lib/python3.6/site-packages/flask/

In [1]:
########Similarity plotting of search
import time
import xml.etree.ElementTree as ET
import numpy as np
import requests
from urllib.parse import quote
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.layouts import column
from bokeh.models import HoverTool, CustomJS, OpenURL, TapTool, Range1d
from bokeh.models.widgets import TextInput, Button
output_notebook()

#Articles that a given article cites
#https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=24923681
#add to end of elink post: &query_key=<key>&WebEnv=<webenv string>
#Parks Stone Pubmed ID 24923681

# UID list. Either a single UID or a comma-delimited list of UIDs may be provided.
# All of the UIDs must be from the database specified by dbfrom. 
# There is no set maximum for the number of UIDs that can be passed to ELink, 
# but if more than about 200 UIDs are to be provided, the request should be made using the HTTP POST method.

ss = "centromere"
syear = "1990"
eyear = "2017"

#similarity score for comparing sets, ie cited articles
#see http://dataconomy.com/2015/04/implementing-the-five-most-popular-similarity-measures-in-python/
def jaccard(x,y):

    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def returnJaccard(ids,cids):
    jarr = np.empty([len(ids),len(ids)])
    for ix,i in enumerate(ids):
        for jx,j in enumerate(ids):
            if(ix>jx):
                jc = jaccard(cids[ix],cids[jx])
                jarr[ix][jx] = jc
                jarr[jx][ix] = jc
            elif(ix==jx):
                #not sure why i had to add this line
                jarr[ix][jx] = 0
    return jarr

### Get ids (PMID) for papers in search
def PMIDsFromSearch(s,sy,ey):
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="+s+"&mindate="+sy+"/01/01&maxdate="+ey+"/12/31&usehistory=y&retmode=json"
    search_r = requests.post(search_url)
    search_data = search_r.json()
    web_env = search_data['esearchresult']['webenv']
    query_key = search_data["esearchresult"]['querykey']
    this_fetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&query_key="+query_key+"&WebEnv="+web_env
    #this_fetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&query_key="+query_key+"&WebEnv="+web_env+"&cmd=neighbor_score"
    #this_fetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&db=pubmed&id=24923681&cmd=neighbor_score"
    
    pre = time.time()
    fetch_r = requests.post(this_fetch)
    print("PMIDs Retrieved:"+str(time.time()-pre))
    #print(fetch_r.text)
    return ET.fromstring(fetch_r.text)



def getCitedFromPMIDXML(r):
    ids = []
    cids = []

    cited_post = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"
    post_vars = {"dbfrom":"pubmed", "linkname":"pubmed_pubmed_refs","id":[]}

    #loop over all articles and grab set cited papers (see cited_rosetta for paper structure)
    for tid in r[0][1].iter('Id'):
        post_vars['id'].append(tid.text)
    
    pre = time.time()
    cited_fetch = requests.post(cited_post, data=post_vars)
    print("References Retrieved:"+str(time.time()-pre))
    croot = ET.fromstring(cited_fetch.text)
    
    for linkset in croot.iter('LinkSet'):
        tlinks = []
        ttid = linkset.find('IdList')[0]
        if(len(list(linkset))>2):
            for link in linkset.find('LinkSetDb').iter('Link'):
                tlid = link[0]
                tlinks.append(tlid.text)
            cids.append(tlinks)
            ids.append(int(ttid.text))
    return ids, cids

#ids = np.empty([len(root[0][1].getchildren()),1])
# def getCitedFromPMIDXML(r):
#     ids = []
#     cids = []

#     cited_xml = ""
#     cited_base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs"
#     cited_post = cited_base

#     #loop over all articles and grab set cited papers (see cited_rosetta for paper structure)
#     for tid in r[0][1].iter('Id'):
#         if((len(cited_post)+len(tid.text)+4)>2083):
#             #print("fetching")
#             cited_fetch = requests.post(cited_post)
#             cited_post = cited_base+"&id="+tid.text
#             croot = ET.fromstring(cited_fetch.text)

#             for linkset in croot.iter('LinkSet'):
#                 tlinks = []
#                 ttid = linkset.find('IdList')[0]
#                 if(len(list(linkset))>2):
#                     for link in linkset.find('LinkSetDb').iter('Link'):
#                         tlid = link[0]
#                         tlinks.append(tlid.text)
#                     cids.append(tlinks)
#                     ids.append(int(ttid.text))
#         else:
#             cited_post += "&id="+tid.text

#     #for grabbing last set of Links
#     cited_fetch = requests.post(cited_post)
#     cited_post = cited_base+"&id="+tid.text
#     croot = ET.fromstring(cited_fetch.text)

#     for linkset in croot.iter('LinkSet'):
#         tlinks = []
#         ttid = linkset.find('IdList')[0]
#         if(len(list(linkset))>2):
#             for link in linkset.find('LinkSetDb').iter('Link'):
#                 tlid = link[0]
#                 tlinks.append(int(tlid.text))
#             cids.append(tlinks)
#             ids.append(int(ttid.text))
#     return ids, cids




########CHANGE CODE BELOW need to grab all ids summary to get the title, authors, pubdate for each
def getPMIDInfo(ids):
    #use full journal name
    titles = []
    dates = []
    authors = []
    journals = []
    
    #https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&version=2.0&id=27656642,24923681
    info_base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    info_data = {"db":"pubmed", "version":"2.0"}
    info_str = ""
    info_post = info_base
    
    #loop over all PMIDs
    for tid in ids:
        info_str += str(tid)+","
    
    info_str = info_str[:-1]
    info_data["id"] = info_str
    #print(info_str)
    
    pre = time.time()
    info_fetch = requests.post(info_post, data=info_data)
    print("Info Retrieved:"+str(time.time()-pre))
    
    croot = ET.fromstring(info_fetch.text)

    for doc in croot[0].iter('DocumentSummary'):
        titles.append(doc.find('Title').text)
        dates.append(doc.find('PubDate').text)
        journals.append(doc.find('FullJournalName').text)
        aus = ""
        for auth in doc.find('Authors').iter('Author'):
            aus += auth.find('Name').text+", "
        authors.append(aus[:-2])
    

    return titles, dates, authors, journals

########CHANGE CODE BELOW need to grab all ids summary to get the title, authors, pubdate for each
# def getPMIDInfo(ids):
#     #use full journal name
#     titles = []
#     dates = []
#     authors = []
#     journals = []
    
#     #https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&version=2.0&id=27656642,24923681
#     info_base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&version=2.0&id="
#     info_post = info_base
    
#     #loop over all PMIDs
#     for tid in ids:
#         if((len(info_post)+len(str(tid))+4)>2083):
            
#             #print("fetching")
#             info_post = info_post[:-1]
#             info_fetch = requests.post(info_post)
#             info_post = info_base+str(tid)+","
#             croot = ET.fromstring(info_fetch.text)

#             #get data from croot
#             for doc in croot[0].iter('DocumentSummary'):
#                 titles.append(doc.find('Title').text)
#                 dates.append(doc.find('PubDate').text)
#                 journals.append(doc.find('FullJournalName').text)
#                 aus = ""
#                 for auth in doc.find('Authors').iter('Author'):
#                     aus += auth.find('Name').text+", "
#                 authors.append(aus[:-2])
                
#         else:
#             info_post += str(tid)+","

#     #for grabbing last set of Links
    
#     info_post = info_post[:-1]
#     info_fetch = requests.post(info_post)
#     croot = ET.fromstring(info_fetch.text)

#     for doc in croot[0].iter('DocumentSummary'):
#         titles.append(doc.find('Title').text)
#         dates.append(doc.find('PubDate').text)
#         journals.append(doc.find('FullJournalName').text)
#         aus = ""
#         for auth in doc.find('Authors').iter('Author'):
#             aus += auth.find('Name').text+", "
#         authors.append(aus[:-2])
#     return titles, dates, authors, journals

def calcTSNE(X):
#     print("SVD calc")
#     #X = jarr
#     svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
#     Xr = svd.fit_transform(X)

    print("TSNE calc")
    tsn = TSNE(n_components=2, random_state=0)
    Y = tsn.fit_transform(X);
    
    return Y

# print("xmax "+str(np.amax(Y[:,0])))
# print("xmin "+str(np.amin(Y[:,0])))
# print("ymax "+str(np.amax(Y[:,1])))
# print("ymin "+str(np.amin(Y[:,1])))
def jaccardSimilarityGraph(si, sy, ey):
    pres = time.time()
    ss = quote(si)
    print("Searching for PMIDs")
    root = PMIDsFromSearch(ss, sy, ey)
    print("Getting Cited PMIDs")
    ids, cids = getCitedFromPMIDXML(root)
    print(str(len(ids))+" Articles Found")
    print("Getting Info of PMIDs")
    titles, dates, authors, journals = getPMIDInfo(ids)
    print("Performing Jaccard comparison")
    pre = time.time()
    jarr = returnJaccard(ids, cids)
    print("Jaccard comparison time:"+str(time.time()-pre))
    #print("min jaccard:"+str(np.amin(jarr)))
    pre = time.time()
    Y = calcTSNE(jarr)
    print("TSNE time:"+str(time.time()-pre))
    print("Total Time:"+str(time.time()-pres))
    
    colors = ['blue']*len(ids)
    alphas = [1]*len(ids)
    source = ColumnDataSource(
            data=dict(
                x=Y[:,0],
                y=Y[:,1],
                PMID=ids,
                titles=titles,
                authors=authors,
                journals=journals,
                dates=dates,
                colors=colors,
                alphas=alphas
            )
        )

    #####max-width IS IMPORTANT FOR PROPER WRAPPING OF TEXT
    hover = HoverTool(
            tooltips="""
            <div>
                <div style="max-width: 400px;">
                    <span style="font-size: 12px; font-weight: bold;">@titles</span>
                </div>
                <div style="max-width: 400px;">
                    <span style="font-size: 12px; color: #966;">@authors</span>
                <div>
                <div style="max-width: 400px;">
                    <span style="font-size: 12px; font-style: italic;">@journals, @dates</span>
                <div style="max-width: 400px;">
                    <span style="font-size: 10px;">PMID</span>
                    <span style="font-size: 10px; color: #696;">@PMID</span>
                </div>
            </div>
            """
        )

    resetCallback = CustomJS(args=dict(source=source), code="""
        var data = source.get('data')
        var titles = data['titles']
        for (i=0; i < titles.length; i++) {
            data.colors[i]='blue'
            data.alphas[i]= 1
        }
        source.trigger('change')
    """)

    textCallback = CustomJS(args=dict(source=source), code="""
        var data = source.get('data')
        var value = cb_obj.get('value')
        var words = value.split(" ")
        for (i=0; i < data.titles.length; i++) {
            data.alphas[i]= 0.3
        }
        for (i=0; i < data.titles.length; i++) {
            for(j=0; j < words.length; j++){
                if (data.titles[i].toLowerCase().indexOf(words[j].toLowerCase()) !== -1) { 
                    if(j == words.length-1){
                        data.colors[i]='orange'
                        data.alphas[i]= 1
                    }
                }else if(data.authors[i].toLowerCase().indexOf(words[j].toLowerCase()) !== -1){
                    if(j == words.length-1){
                        data.colors[i]='orange'
                        data.alphas[i]= 1
                    }
                }else if(data.journals[i].toLowerCase().indexOf(words[j].toLowerCase()) !== -1){
                    if(j == words.length-1){
                        data.colors[i]='orange'
                        data.alphas[i]= 1
                    }
                }else{
                    break
                }
            }
        }
        source.trigger('change')
    """)


    TOOLS = 'pan,wheel_zoom,tap,reset'
    p = figure(plot_width=900, plot_height=400, title="'"+si+"' tSNE similarity", tools=[TOOLS,hover], active_scroll='wheel_zoom')

    p.circle('x', 'y',fill_color='colors', fill_alpha='alphas', size=12, source=source)

    #formatting plot
    p.xaxis.axis_label = "Hover to view publication info, Click to open Pubmed link"
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
    p.yaxis.major_label_text_font_size = '0pt'
    left, right, bottom, top = np.amin(Y[:,0])*1.1, np.amax(Y[:,0])*1.1, np.amin(Y[:,1])*1.1, np.amax(Y[:,1])*1.1
    p.x_range=Range1d(left, right)
    p.y_range=Range1d(bottom, top)


    url = "https://www.ncbi.nlm.nih.gov/pubmed/@PMID"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)


    word_input = TextInput(title="Search for term within dataset (highlight matching pubs)", callback=textCallback)
    reset = Button(label="Reset Matching Pubs", callback=resetCallback, width=150)

    layout = column(word_input,reset, p)

    show(layout)

    
jaccardSimilarityGraph(ss,syear,eyear)

# print(jarr[:5])
# print(ids[:5])
# print(cids[:5])
# print(titles[:5])
# print(authors[:5])
# print(dates[:5])
# print(journals[:5])


# cited_fetch = requests.post(cited_post)
# cited_xml += cited_fetch.text
# f = open("cited_"+ss+".xml", 'w')
# f.write(cited_xml)
# f.close()

Loading BokehJS ...

Searching for PMIDs
PMIDs Retrieved:4.715541124343872
Getting Cited PMIDs
References Retrieved:69.86473393440247
4130 Articles Found
Getting Info of PMIDs
Info Retrieved:1.8073039054870605
Performing Jaccard comparison
Jaccard comparison time:150.32236003875732
TSNE calc
TSNE time:155.04069781303406
Total Time:385.36028695106506


In [ ]:
print(jaccard(cids[ids.index(27895053)],cids[ids.index(27180908)]))

In [8]:
#############ASYCIO PubByDate

from aiohttp import ClientSession
import asyncio
import urllib
from urllib.parse import quote
#import concurrent.futures
#import requests

import json
from bokeh.sampledata import us_states
from bokeh.plotting import figure, show, output_notebook


years = list(range(1975, 2017))

async def fetchYears(url, session):
    async with session.get(url) as response:
        return await response.text()

###add years correlating with responses############
async def runYears(yrs, ss):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for y in yrs:
            tu = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="+ss+"&mindate="+str(y)+"/01/01&maxdate="+str(y)+"/12/31&usehistory=y&retmode=json"
            task = asyncio.ensure_future(fetchYears(tu, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        
        # you now have all response bodies in this variable
        #responses can be converted to json, originally were strings
        #print(json.loads(responses[0]))
    return responses

# gets data from each state while string : ss=searchstring
def getYears(ss):
    
    yearcounts = []
    #start threads and create queue of URLs
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(runYears(years,ss))
    res = loop.run_until_complete(future)
    #print(res)
    for idx, y in enumerate(years):
        search_data = json.loads(res[idx])
        #webenv = search_data["esearchresult"]['webenv']
        total_records = int(search_data["esearchresult"]['count'])
        yearcounts.append(total_records) 
        #print(total_records)
    return yearcounts
        

def yearGraph(si):
    ss = quote(si)
    p = figure(title="Articles containing: "+si, plot_width=400, plot_height=400, x_axis_label="Year", y_axis_label="Total Articles")

    ##FOR SEARCHING YOU WILL NEED TO ESCAPE SPACES AND SPECIAL CHARS
    yearcounts = getYears(ss)
#     for state in us_states:
#         print(state)
#         print(us_states[state]["count"])

    # unnormalized to money version
    p.line(years, yearcounts, color='blue')

    #show(p)
    return p

#p, p2 = stateGraph("prostate")
#show(p)
#show(p2)
# stateGraph("lung")
# stateGraph("breast")



In [9]:
#########ASYNCIO version of StateGraph

# StateGraph -  used to pull data from pubmed through an api
from aiohttp import ClientSession
import asyncio
import urllib
from urllib.parse import quote
#import concurrent.futures
#import requests

import json
from bokeh.sampledata import us_states
from bokeh.plotting import figure, show, output_notebook
import pickle

dataDir = "./static/"
moneyFile = "FundingPerState2016.pkl"

# affiliation = AD
searchField = "[AD]"
us_states = us_states.data.copy()
del us_states["HI"]
del us_states["AK"]

state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]

async def fetchStates(url, session):
    async with session.get(url) as response:
        return await response.text()

###add states correlating with responses############
async def runStates(states, ss):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for state in states:
            tu = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=" + state + searchField+"+AND+" +ss+"&mindate=2012/01/01&maxdate=2016/12/31&usehistory=y&retmode=json"
            task = asyncio.ensure_future(fetchStates(tu, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        
        # you now have all response bodies in this variable
        #responses can be converted to json, originally were strings
        #print(json.loads(responses[0]))
    return responses

# gets data from each state while string : ss=searchstring
def getStates(ss):
    #start threads and create queue of URLs
    loop = asyncio.get_event_loop()
    states = [us_states[state]["name"] for state in us_states]
    future = asyncio.ensure_future(runStates(states,ss))
    res = loop.run_until_complete(future)
    #print(res)
    for idx, state in enumerate(us_states):
        search_data = json.loads(res[idx])
        #webenv = search_data["esearchresult"]['webenv']
        total_records = int(search_data["esearchresult"]['count'])
        us_states[state]["count"] = total_records
        #print(total_records)
        

def stateGraph(si):
    ss = quote(si)
    p = figure(title="Publications containing: " + si,
               toolbar_location="left", plot_width=800, plot_height=510)
    p2 = figure(title="Publications containing: "+si+" (Normalized by funding)",
                toolbar_location="left", plot_width=800, plot_height=510)
    p.xaxis.visible = False
    p.xgrid.visible = False
    p.yaxis.visible = False
    p.ygrid.visible = False
    
    p2.xaxis.visible = False
    p2.xgrid.visible = False
    p2.yaxis.visible = False
    p2.ygrid.visible = False

    ##FOR SEARCHING YOU WILL NEED TO ESCAPE SPACES AND SPECIAL CHARS
    getStates(ss)
#     for state in us_states:
#         print(state)
#         print(us_states[state]["count"])

    # unnormalized to money version
    state_counts = [us_states[code]["count"] for code in us_states]
    state_counts_norm = state_counts
    max_state_counts = max(state_counts)
    if(max_state_counts > 0):
        state_counts = [x / max_state_counts for x in state_counts]
    else:
        state_counts = [x for x in state_counts]

    # normalized to money
    fbs = pickle.load(open(dataDir + moneyFile, "rb"))
    state_counts_norm = [us_states[code]["count"] / fbs[us_states[code]["name"]] for code in us_states]
    max_state_counts_norm = max(state_counts_norm)
    if(max_state_counts_norm > 0):
        state_counts_norm = [x / max_state_counts_norm for x in state_counts_norm]
    else:
        state_counts_norm = [x for x in state_counts_norm]
    p.patches(state_xs, state_ys, fill_color="#377BA8", fill_alpha=state_counts,
              line_color="#884444", line_width=1.5)

    p2.patches(state_xs, state_ys, fill_color="#377BA8", fill_alpha=state_counts_norm,
               line_color="#884444", line_width=1.5)

    #show(p)
    #show(p2)
    return p, p2

#p, p2 = stateGraph("prostate")
#show(p)
#show(p2)
# stateGraph("lung")
# stateGraph("breast")



In [ ]:
###plotting subset of 

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.layouts import column
from bokeh.models import HoverTool, CustomJS, OpenURL, TapTool, Range1d
from bokeh.models.widgets import TextInput, Button
#output_notebook()

def calcTSNE(X)
    print("SVD calc")
    #X = jarr
    svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
    Xr = svd.fit_transform(X)

    print("TSNE calc")
    tsn = TSNE(n_components=2, random_state=0)
    Y = tsn.fit_transform(Xr);
    
    return Y

# print("xmax "+str(np.amax(Y[:,0])))
# print("xmin "+str(np.amin(Y[:,0])))
# print("ymax "+str(np.amax(Y[:,1])))
# print("ymin "+str(np.amin(Y[:,1])))
def similarityGraph(X,ss,ids,titles,authors,journals,dates)
    
    Y = calcTSNE(X)
    
    colors = ['blue']*len(ids)
    source = ColumnDataSource(
            data=dict(
                x=Y[:,0],
                y=Y[:,1],
                PMID=ids,
                titles=titles,
                authors=authors,
                journals=journals,
                dates=dates,
                colors=colors
            )
        )

    #####max-width IS IMPORTANT FOR PROPER WRAPPING OF TEXT
    hover = HoverTool(
            tooltips="""
            <div>
                <div style="max-width: 400px;">
                    <span style="font-size: 12px; font-weight: bold;">@titles</span>
                </div>
                <div style="max-width: 400px;">
                    <span style="font-size: 12px; color: #966;">@authors</span>
                <div>
                <div style="max-width: 400px;">
                    <span style="font-size: 12px; font-style: italic;">@journals, @dates</span>
                <div style="max-width: 400px;">
                    <span style="font-size: 10px;">PMID</span>
                    <span style="font-size: 10px; color: #696;">@PMID</span>
                </div>
            </div>
            """
        )

    resetCallback = CustomJS(args=dict(source=source), code="""
        var data = source.get('data')
        var titles = data['titles']
        for (i=0; i < titles.length; i++) {
            data.colors[i]='blue'
        }
        source.trigger('change')
    """)

    textCallback = CustomJS(args=dict(source=source), code="""
        var data = source.get('data')
        var value = cb_obj.get('value')
        var titles = data['titles']
        var authors = data['authors']
        var journals = data['journals']
        var words = value.split(" ")
        for (i=0; i < titles.length; i++) {
            for(j=0; j < words.length; j++){
                if (titles[i].toLowerCase().indexOf(words[j].toLowerCase()) !== -1) { 
                    if(j == words.length-1){
                        data.colors[i]='orange' 
                    }
                }else if(authors[i].toLowerCase().indexOf(words[j].toLowerCase()) !== -1){
                    if(j == words.length-1){
                        data.colors[i]='orange' 
                    }
                }else if(journals[i].toLowerCase().indexOf(words[j].toLowerCase()) !== -1){
                    if(j == words.length-1){
                        data.colors[i]='orange' 
                    }
                }else{
                    break
                }
            }
        }
        source.trigger('change')
    """)


    TOOLS = 'pan,wheel_zoom,tap,reset'
    p = figure(plot_width=900, plot_height=400, title="'"+ss+"' tSNE similarity", tools=[TOOLS,hover], active_scroll='wheel_zoom')

    p.circle('x', 'y',fill_color='colors', size=12, source=source)

    #formatting plot
    p.xaxis.axis_label = "Hover to view publication info, Click to open Pubmed link"
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
    p.yaxis.major_label_text_font_size = '0pt'
    left, right, bottom, top = np.amin(Y[:,0])*1.1, np.amax(Y[:,0])*1.1, np.amin(Y[:,1])*1.1, np.amax(Y[:,1])*1.1
    p.x_range=Range1d(left, right)
    p.y_range=Range1d(bottom, top)


    url = "https://www.ncbi.nlm.nih.gov/pubmed/@PMID"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)


    word_input = TextInput(title="Search for term within dataset (highlight matching pubs)", callback=textCallback)
    reset = Button(label="Reset Matching Pubs", callback=resetCallback, width=150)

    layout = column(word_input,reset, p)

    show(layout)
    #return layout

#Plot.scatter(Y[:,0], Y[:,1], 20);
#Plot.show();
#print(Xr[:5])
#print(svd.explained_variance_ratio_.sum()) 
#Y = tsne(X, 2, 50, 20.0);
#Plot.scatter(Y[:,0], Y[:,1], 20, labels);
#Plot.show();

In [ ]:
#
#  tsne.py
#
# Implementation of t-SNE in Python. The implementation was tested on Python 2.7.10, and it requires a working
# installation of NumPy. The implementation comes with an example on the MNIST dataset. In order to plot the
# results of this example, a working installation of matplotlib is required.
#
# The example can be run by executing: `ipython tsne.py`
#
#
#  Created by Laurens van der Maaten on 20-12-08.
#  Copyright (c) 2008 Tilburg University. All rights reserved.

import numpy as Math
import pylab as Plot

def Hbeta(D = Math.array([]), beta = 1.0):
	"""Compute the perplexity and the P-row for a specific value of the precision of a Gaussian distribution."""

	# Compute P-row and corresponding perplexity
	P = Math.exp(-D.copy() * beta);
	sumP = sum(P);
	H = Math.log(sumP) + beta * Math.sum(D * P) / sumP;
	P = P / sumP;
	return H, P;


def x2p(X = Math.array([]), tol = 1e-5, perplexity = 30.0):
	"""Performs a binary search to get P-values in such a way that each conditional Gaussian has the same perplexity."""

	# Initialize some variables
	print("Computing pairwise distances...")
	(n, d) = X.shape;
	sum_X = Math.sum(Math.square(X), 1);
	D = Math.add(Math.add(-2 * Math.dot(X, X.T), sum_X).T, sum_X);
	P = Math.zeros((n, n));
	beta = Math.ones((n, 1));
	logU = Math.log(perplexity);

	# Loop over all datapoints
	for i in range(n):

		# Print progress
		if i % 500 == 0:
			print("Computing P-values for point ", i, " of ", n, "...")

		# Compute the Gaussian kernel and entropy for the current precision
		betamin = -Math.inf;
		betamax =  Math.inf;
		Di = D[i, Math.concatenate((Math.r_[0:i], Math.r_[i+1:n]))];
		(H, thisP) = Hbeta(Di, beta[i]);

		# Evaluate whether the perplexity is within tolerance
		Hdiff = H - logU;
		tries = 0;
		while Math.abs(Hdiff) > tol and tries < 50:

			# If not, increase or decrease precision
			if Hdiff > 0:
				betamin = beta[i].copy();
				if betamax == Math.inf or betamax == -Math.inf:
					beta[i] = beta[i] * 2;
				else:
					beta[i] = (beta[i] + betamax) / 2;
			else:
				betamax = beta[i].copy();
				if betamin == Math.inf or betamin == -Math.inf:
					beta[i] = beta[i] / 2;
				else:
					beta[i] = (beta[i] + betamin) / 2;

			# Recompute the values
			(H, thisP) = Hbeta(Di, beta[i]);
			Hdiff = H - logU;
			tries = tries + 1;

		# Set the final row of P
		P[i, Math.concatenate((Math.r_[0:i], Math.r_[i+1:n]))] = thisP;

	# Return final P-matrix
	print("Mean value of sigma: ", Math.mean(Math.sqrt(1 / beta)));
	return P;


def pca(X = Math.array([]), no_dims = 50):
	"""Runs PCA on the NxD array X in order to reduce its dimensionality to no_dims dimensions."""

	print("Preprocessing the data using PCA...")
	(n, d) = X.shape;
	X = X - Math.tile(Math.mean(X, 0), (n, 1));
	(l, M) = Math.linalg.eig(Math.dot(X.T, X));
	Y = Math.dot(X, M[:,0:no_dims]);
	return Y;


def tsne(X = Math.array([]), no_dims = 2, initial_dims = 50, perplexity = 30.0):
	"""Runs t-SNE on the dataset in the NxD array X to reduce its dimensionality to no_dims dimensions.
	The syntaxis of the function is Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array."""

	# Check inputs
	if isinstance(no_dims, float):
		print("Error: array X should have type float.");
		return -1;
	if round(no_dims) != no_dims:
		print("Error: number of dimensions should be an integer.");
		return -1;

	# Initialize variables
	X = pca(X, initial_dims).real;
	(n, d) = X.shape;
	max_iter = 1000;
	initial_momentum = 0.5;
	final_momentum = 0.8;
	eta = 500;
	min_gain = 0.01;
	Y = Math.random.randn(n, no_dims);
	dY = Math.zeros((n, no_dims));
	iY = Math.zeros((n, no_dims));
	gains = Math.ones((n, no_dims));

	# Compute P-values
	P = x2p(X, 1e-5, perplexity);
	P = P + Math.transpose(P);
	P = P / Math.sum(P);
	P = P * 4;									# early exaggeration
	P = Math.maximum(P, 1e-12);

	# Run iterations
	for iter in range(max_iter):

		# Compute pairwise affinities
		sum_Y = Math.sum(Math.square(Y), 1);
		num = 1 / (1 + Math.add(Math.add(-2 * Math.dot(Y, Y.T), sum_Y).T, sum_Y));
		num[range(n), range(n)] = 0;
		Q = num / Math.sum(num);
		Q = Math.maximum(Q, 1e-12);

		# Compute gradient
		PQ = P - Q;
		for i in range(n):
			dY[i,:] = Math.sum(Math.tile(PQ[:,i] * num[:,i], (no_dims, 1)).T * (Y[i,:] - Y), 0);

		# Perform the update
		if iter < 20:
			momentum = initial_momentum
		else:
			momentum = final_momentum
		gains = (gains + 0.2) * ((dY > 0) != (iY > 0)) + (gains * 0.8) * ((dY > 0) == (iY > 0));
		gains[gains < min_gain] = min_gain;
		iY = momentum * iY - eta * (gains * dY);
		Y = Y + iY;
		Y = Y - Math.tile(Math.mean(Y, 0), (n, 1));

		# Compute current value of cost function
		if (iter + 1) % 10 == 0:
			C = Math.sum(P * Math.log(P / Q));
			print("Iteration ", (iter + 1), ": error is ", C)

		# Stop lying about P-values
		if iter == 100:
			P = P / 4;

	# Return solution
	return Y;


if __name__ == "__main__":
    print("Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset.")
    print("Running example on 2,500 MNIST digits...")
    X = Math.loadtxt("mnist2500_X.txt");
    #print(X.shape)
    labels = Math.loadtxt("mnist2500_labels.txt");
    Y = tsne(X, 2, 50, 20.0);
    Plot.scatter(Y[:,0], Y[:,1], 20, labels);
    Plot.show();


In [ ]:
######Get state data based on Json search, prototype code, real implementation above

from urllib.parse import quote
import requests
import json

def getPubData(si):
    ss = quote(si)
    y1 = 2016
    y2 = 2017
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="+ss+"&mindate="+str(y1)+"/01/01&maxdate="+str(y2)+"/12/31&usehistory=y&retmode=json"
    search_r = requests.post(search_url)
    search_data = search_r.json()
    webenv = search_data["esearchresult"]['webenv']
    total_records = int(search_data["esearchresult"]['count'])
    fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&retmax=9999&query_key=1&webenv="+webenv

    #must grab data in chunks, limited to 10000 per query
    allpubs = ""
    for i in range(0, total_records, 10000):
        this_fetch = fetch_url+"&retstart="+str(i)
        print("Getting this URL: "+this_fetch)
        fetch_r = requests.post(this_fetch)
        allpubs += fetch_r.text
    print(type(fetch_r))
    f = open("pubmedsearchdata.xml", 'w')
    f.write(allpubs)
    f.close()
    print("Number of records found :"+str(total_records))
    
getPubData("cas9")

In [ ]:
####ASYNCIO CountryGraph

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.sampledata.sample_geojson import geojson
from bokeh.models import Range1d, GeoJSONDataSource
import pickle
import json
output_notebook()


with open('./static/countries.json') as c:
        countriesjson = (c.read())
        
countriesgj = GeoJSONDataSource(geojson=countriesjson)

p = figure(width = 1000, height=500, title='World Countries', x_axis_label='Longitude', y_axis_label='Latitude')

p.patches(xs= 'xs', ys='ys', source=countriesgj, fill_color="#FFFFFF", line_color="#333333", line_width=1.5)
    
    
p.x_range = Range1d(start = -180, end = 180)
p.y_range = Range1d(start = -90, end = 90)
show(p)

In [ ]:
##### Get country data and save to pandas
import pandas as pd
import requests
import pickle

countries = requests.get('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json').json()


countryObject = {}
for country in countries['features']:
    countryObject[country['properties']['name']] = {
        'x': [x[0] for x in country['geometry']['coordinates'][0]],
        'y': [x[1] for x in country['geometry']['coordinates'][0]],
    }
countriesjson = open("countriesdf.pkl", 'w')


countrydf = pd.DataFrame(countryObject)
countryout = open("countriesdf.pkl", 'wb')
pickle.dump(countrydf, countryout)
countryout.close()

In [ ]:
########Javascript bokeh update example

from bokeh.io import vform
from bokeh.models import HoverTool, CustomJS
from bokeh.models.widgets import TextInput
from bokeh.plotting import output_file, figure, show, ColumnDataSource

output_file("plot.html")

words = ["werner", "herbert", "klaus"]
x, y = [1,2,3], [1,2,3]
color = ['green', 'blue', 'red']

source = ColumnDataSource(data=dict(x=x, y=y, words=words, color=color))

hover = HoverTool(tooltips=[("word", "@words")])

p = figure(plot_height=600, plot_width=800, title="word2vec", tools=[hover])
p.circle('x','y', radius=0.1, fill_color='color', source=source, line_color=None)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.get('data')
    var value = cb_obj.get('value')
    var words = data['words']
    for (i=0; i < words.length; i++) {
        if ( words[i]==value ) { data.color[i]='yellow' }
    }
    source.trigger('change')
""")

word_input = TextInput(value="word", title="Point out a word", callback=callback)

layout = vform(word_input, p)

show(layout)